In [1]:
# -----------------------------
# Cell 0: Environment Setup
# -----------------------------
import sys
import os

# Add project root to Python path
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root:", PROJECT_ROOT)

# -----------------------------
# Imports
# -----------------------------
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import cv2

# -----------------------------
# CUDA Check
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Project root: d:\u-net-biomedical-flow-project
CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
# -----------------------------
# Cell 1: U-Net Model
# -----------------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)


class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.enc1 = DoubleConv(1, 32)
        self.enc2 = DoubleConv(32, 64)
        self.enc3 = DoubleConv(64, 128)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = DoubleConv(128, 256)

        self.up3 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec3 = DoubleConv(256, 128)

        self.up2 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec2 = DoubleConv(128, 64)

        self.up1 = nn.ConvTranspose2d(64, 32, 2, 2)
        self.dec1 = DoubleConv(64, 32)

        self.final = nn.Conv2d(32, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))

        b = self.bottleneck(self.pool(e3))

        d3 = self.up3(b)
        d3 = self.dec3(torch.cat([d3, e3], dim=1))

        d2 = self.up2(d3)
        d2 = self.dec2(torch.cat([d2, e2], dim=1))

        d1 = self.up1(d2)
        d1 = self.dec1(torch.cat([d1, e1], dim=1))

        return torch.sigmoid(self.final(d1))


In [3]:
# -----------------------------
# Cell 2: Dataset (Pylance-safe)
# -----------------------------
class UltrasoundDataset(Dataset):
    def __init__(self, image_dir: str):
        self.image_dir = image_dir
        self.files = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.files[idx])

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # 🔒 Explicit None check (fixes cv2.resize warning)
        if img is None:
            raise RuntimeError(f"Failed to load image: {img_path}")

        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32) / 255.0

        # 🔒 NaN / Inf protection
        img = np.nan_to_num(img, nan=0.0, posinf=1.0, neginf=0.0)
        img = np.clip(img, 0.0, 1.0)

        x = torch.from_numpy(img).unsqueeze(0)
        y = x.clone()  # autoencoder-style target

        return x, y


In [4]:
# -----------------------------
# Cell 3: DataLoaders
# -----------------------------
DATASET_PATH = "../data/images"  # update if needed

dataset = UltrasoundDataset(DATASET_PATH)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_ds,
    batch_size=8,
    shuffle=True,
    num_workers=0,      # Windows-safe
    pin_memory=False
)

val_loader = DataLoader(
    val_ds,
    batch_size=8,
    shuffle=False,
    num_workers=0,
    pin_memory=False
)

print("Train samples:", len(train_ds))
print("Val samples:", len(val_ds))


Train samples: 1262
Val samples: 316


In [5]:
# -----------------------------
# Cell 4: Training Setup
# -----------------------------
model = UNet().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

use_amp = False  # 🔥 DISABLED for stability

MODEL_PATH = "../backend/models/unet_model.pth"
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)


In [6]:
# -----------------------------
# Cell 5: Training Loop
# -----------------------------
best_val_loss = float("inf")
epochs = 12

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        out = model(x)
        loss = criterion(out, y)

        if torch.isnan(loss):
            print("⚠️ NaN loss detected, skipping batch")
            continue

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= max(1, len(train_loader))

    # ---------- Validation ----------
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)

            out = model(x)
            loss = criterion(out, y)

            if not torch.isnan(loss):
                val_loss += loss.item()

    val_loss /= max(1, len(val_loader))

    # ---------- LR Schedule ----------
    if epoch == 6:
        for g in optimizer.param_groups:
            g["lr"] = 3e-4
        print("🔽 Learning rate reduced to 3e-4")

    # ---------- Save Best ----------
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
        print("✅ Best model saved")

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f}"
    )


✅ Best model saved
Epoch 1/12 | Train Loss: 0.0111 | Val Loss: 0.0004
✅ Best model saved
Epoch 2/12 | Train Loss: 0.0003 | Val Loss: 0.0002
✅ Best model saved
Epoch 3/12 | Train Loss: 0.0001 | Val Loss: 0.0001
✅ Best model saved
Epoch 4/12 | Train Loss: 0.0001 | Val Loss: 0.0001
✅ Best model saved
Epoch 5/12 | Train Loss: 0.0001 | Val Loss: 0.0001
✅ Best model saved
Epoch 6/12 | Train Loss: 0.0001 | Val Loss: 0.0000
🔽 Learning rate reduced to 3e-4
✅ Best model saved
Epoch 7/12 | Train Loss: 0.0000 | Val Loss: 0.0000
✅ Best model saved
Epoch 8/12 | Train Loss: 0.0000 | Val Loss: 0.0000
✅ Best model saved
Epoch 9/12 | Train Loss: 0.0000 | Val Loss: 0.0000
✅ Best model saved
Epoch 10/12 | Train Loss: 0.0000 | Val Loss: 0.0000
✅ Best model saved
Epoch 11/12 | Train Loss: 0.0000 | Val Loss: 0.0000
✅ Best model saved
Epoch 12/12 | Train Loss: 0.0000 | Val Loss: 0.0000
